# Part1 : Data wrangling

## Data Download and pre-processing

In [151]:
import requests
from robobrowser import RoboBrowser
from bs4 import BeautifulSoup
import re
from lxml import html
import pathlib
import time
import zipfile
from io import BytesIO
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import config
import glob
config.form['accept']

'Yes'

In [159]:
filepath = './zipfile'
#creating the folder for zipfiles
pathlib.Path(filepath).mkdir(parents=True, exist_ok=True) 

#account parameters
username = 'gupta.h@husky.neu.edu'
password = 'puVKND@V'

#Intializing browser
br = RoboBrowser()
#open website
br.open('https://freddiemac.embs.com/FLoan/secure/login.php?pagename=download')  

#Getting form from browser
form = br.get_form()
form['username'] = username
form['password'] = password
br.submit_form(form)

#accept the form
form = br.get_form()
form['accept'] = 'Yes'
form['action'] = 'acceptTandC'
form['acceptSubmit'] = 'Continue'
br.submit_form(form)
links = br.find_all('a')
br.session.cookies

<RequestsCookieJar[Cookie(version=0, name='PHPSESSID', value='vl7khp10st32goaun1o2j61sb0', port=None, port_specified=False, domain='freddiemac.embs.com', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=True, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)]>

In [160]:
cookie = br.session.cookies.get_dict()
cookie

{'PHPSESSID': 'vl7khp10st32goaun1o2j61sb0'}

In [161]:
for link in links:
    regex = re.findall('sample_\d+.zip', str(link))
    if(regex and (int)(re.findall('\d+', str(regex))[0]) > 2005):
        url = "https://freddiemac.embs.com/FLoan/Data/" + link.get('href')
        print(url)
        year = regex[0].split('_')[1][:4]
        print(year)
        content = requests.get(url, cookies = cookie) 
        zf = zipfile.ZipFile(BytesIO(content.content))
        
        #Getting txt files from the zipfiles
        for name in zf.namelist():
            
            #Reading each files
            df = pd.read_csv(zf.open(name), sep= '|', header = None)
            
            if name.split('_')[1] == 'orig':
                #Converting into csv each year
                df.to_csv(filepath + '/' + str(name.split('.')[0]) + '.csv', mode = 'a', index= False, header = None)
            else:
                df.to_csv(filepath + '/' + str(name.split('.')[0]) + '.csv', mode = 'a', index= False, header = None)

https://freddiemac.embs.com/FLoan/Data/download.php?f=sample_2006&s=33286483
2006
https://freddiemac.embs.com/FLoan/Data/download.php?f=sample_2007&s=31029360
2007
https://freddiemac.embs.com/FLoan/Data/download.php?f=sample_2008&s=25243310
2008
https://freddiemac.embs.com/FLoan/Data/download.php?f=sample_2009&s=29742643
2009
https://freddiemac.embs.com/FLoan/Data/download.php?f=sample_2010&s=29502562
2010
https://freddiemac.embs.com/FLoan/Data/download.php?f=sample_2011&s=27895583
2011
https://freddiemac.embs.com/FLoan/Data/download.php?f=sample_2012&s=31533051
2012
https://freddiemac.embs.com/FLoan/Data/download.php?f=sample_2013&s=26626765
2013
https://freddiemac.embs.com/FLoan/Data/download.php?f=sample_2014&s=20459253
2014
https://freddiemac.embs.com/FLoan/Data/download.php?f=sample_2015&s=17376778
2015
https://freddiemac.embs.com/FLoan/Data/download.php?f=sample_2016&s=12007433
2016
https://freddiemac.embs.com/FLoan/Data/download.php?f=sample_2017&s=4621125
2017


# Summary

In [162]:
filepath = './zipfile'
allfiles = glob.glob(filepath + "/*.csv")
print(allfiles)
dfs_origin = []
dfs_performance = []
for files in allfiles:
    print(files)
    if files.split('_')[1] == 'orig':
        print(files.split('_')[1])
        origin_files = pd.read_csv(files ,header = None, index_col= None)
        origin_files.drop(origin_files.columns[25], axis= 1, inplace = True)
        origin_files.columns = ['CREDIT SCORE', 'FIRST PAYMENT DATE', 'FIRST TIME HOMEBUYER FLAG', 'MATURITY DATE', 'MSA', 
              'MORTGAGE INSURANCE PERCENTAGE', 'NUMBER OF UNITS', 'OCCUPANCY STATUS', 'ORIGINAL COMBINED LOAN-TO-VALUE',
             'ORIGINAL DEBT-TO-INCOME (DTI) RATIO', 'ORIGINAL UPB', 'ORIGINAL LOAN-TO-VALUE (LTV)', 
             'ORIGINAL INTEREST RATE', 'CHANNEL', 'PREPAYMENT PENALTY MORTGAGE (PPM) FLAG', 'PRODUCT TYPE', 
                    'PROPERTY STATE', 'PROPERTY TYPE', 'POSTAL CODE', 'LOAN SEQUENCE NUMBER', 'LOAN PURPOSE', 
                    'ORIGINAL LOAN TERM', 'NUMBER OF BORROWERS', 'SELLER NAME', 'SERVICER NAME']
        summmary_files= origin_files.describe()
        summmary_files.to_csv(filepath + "/summary_"+ str(files.split('\\')[1])) 
        dfs_origin.append(origin_files)
    
    else:
        print(files.split('_')[1])
        performance_files = pd.read_csv(files ,header = None, index_col= None)
        performance_files.columns = ['LOAN SEQUENCE NUMBER', 'MONTHLY REPORTING PERIOD', 'CURRENT ACTUAL UPB', 
                                     'CURRENT LOAN DELINQUENCY STATUS', 'LOAN AGE', 'REMAINING MONTHS TO LEGAL MATURITY', 
                                     'REPURCHASE FLAG', 'MODIFICATION FLAG', 'ZERO BALANCE CODE','ZERO BALANCE EFFECTIVE DATE',
                                     'CURRENT INTEREST RATE', 'CURRENT DEFERRED UPB', 'DUE DATE OF LAST PAID INSTALLMENT (DDLPI)',
                                     'MI RECOVERIES', 'NET SALES PROCEEDS', 'NON MI RECOVERIES', 'EXPENSES', 'LEGAL COSTS', 
                                     'MAINTENANCE AND PRESERVATION COSTS', 'TAXES AND INSURANCE', 'MISCELLANEOUS EXPENSES', 
                                     'ACTUAL LOSS CALCULATION', 'MODIFICATION COST', 'STEP MODIFICATION FLAG', 'DEFERRED PAYMENT MODIFICATION',
                                     'ESTIMATED LOAN TO VALUE (ELTV)']
        summary_perf = performance_files.describe()
        summary_perf.to_csv(filepath + "/summary" + str(files.split('\\')[1]))
        dfs_performance.append(performance_files)

['./zipfile\\sample_orig_2006.csv', './zipfile\\sample_orig_2007.csv', './zipfile\\sample_orig_2008.csv', './zipfile\\sample_orig_2009.csv', './zipfile\\sample_orig_2010.csv', './zipfile\\sample_orig_2011.csv', './zipfile\\sample_orig_2012.csv', './zipfile\\sample_orig_2013.csv', './zipfile\\sample_orig_2014.csv', './zipfile\\sample_orig_2015.csv', './zipfile\\sample_orig_2016.csv', './zipfile\\sample_orig_2017.csv', './zipfile\\sample_svcg_2006.csv', './zipfile\\sample_svcg_2007.csv', './zipfile\\sample_svcg_2008.csv', './zipfile\\sample_svcg_2009.csv', './zipfile\\sample_svcg_2010.csv', './zipfile\\sample_svcg_2011.csv', './zipfile\\sample_svcg_2012.csv', './zipfile\\sample_svcg_2013.csv', './zipfile\\sample_svcg_2014.csv', './zipfile\\sample_svcg_2015.csv', './zipfile\\sample_svcg_2016.csv', './zipfile\\sample_svcg_2017.csv']
./zipfile\sample_orig_2006.csv
orig
./zipfile\sample_orig_2007.csv
orig
./zipfile\sample_orig_2008.csv
orig
./zipfile\sample_orig_2009.csv
orig
./zipfile\sampl